# Implementações de Métricas Utilizadas na Clusterização

In [3]:
from sklearn.datasets import load_wine
from sklearn.decomposition import PCA, FastICA
from sklearn.manifold import Isomap, TSNE
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold, train_test_split as tts
from sklearn import metrics
from scipy.spatial.distance import cdist
from sklearn.cluster import KMeans
from sklearn.cluster import AffinityPropagation
from sklearn.utils import shuffle
from scipy.spatial import distance
from sklearn.model_selection import LeaveOneOut
from math import pow
from random import randint
from scipy import stats
from IPython.display import HTML, display

### WB Index

De acordo com o artigo citado, essa métrica pode ser utilizada tanto para averiguar o número ótimo de clusters em um dataset quanto para avaliar o agrupamento feito por um algorotmo. 

O melhor valor para essa métrica é o menor possível.

Referência: http://cs.joensuu.fi/sipu/pub/qinpei-thesis.pdf, página 22.

#### Implementação

In [2]:
def WBIndex(x, y, centroides):
    ssw = sumSquaresWithin(x, y, centroides)
    ssb = sumSquaresBetween(x, y, centroides)
    return  (len(x) * (ssw/ssb))

def sumSquaresWithin(x, y, centroides):
    
    distanciasAoQuadrado = []
    
    #Primeiro passando de cluster em cluster
    for clusterAtual in range(0, len(centroides)):
        
        #Pra cada cluster eu pego todos os pontos dele
        for index, clusterPontoAtual in enumerate(y):
            if clusterAtual == clusterPontoAtual:
                #Se chegou aqui então eu estou em um ponto do cluster atual
                d = x[index] - centroides[clusterAtual]
                distanciasAoQuadrado.append(d * d)
    
    #Aqui já tenho o vetor distanciasAoQuadrado completo com todos os 
    #pontos em relação aos seus centroides, é só retornar a soma de tudo isso
    distanciasAoQuadrado = np.array(distanciasAoQuadrado)    
    
    return np.sum(distanciasAoQuadrado)

def sumSquaresBetween(x, y, centroides):
    
    #Primeiro, tenho que achar o X médio de TODOS os pontos 
    #(embaixo da equação 3.9 (pág 24) tem uma representação visual desse x médio)
    xMedio = []
    for dimensaoAtual in range(0, x.shape[1]):
        xMedio.append(np.mean(x[:, dimensaoAtual]))
    xMedio = np.array(xMedio)
    
    #Beleza, agr já tenho o xMedio, é só aplicar a equação 3.8 e somar tudo como mostra a equação 3.9
    bs = []
    
    for clusterAtual in range(0, len(centroides)):
        #Preciso saber a quantidade de pontos que existem no cluster atual
        qtdPontosClusterAtual = 0
        for clusterPontoAtual in y:
            if clusterPontoAtual == clusterAtual:
                qtdPontosClusterAtual += 1
        
        #agora aplico a equação 3.8...
        d = centroides[clusterAtual] - xMedio
        bs.append(qtdPontosClusterAtual * (d * d))
        
    #...e somo todos os B's como mostra a equação 3.9
    bs = np.array(bs)
    
    return np.sum(bs)

#### Exemplo de uso

In [6]:
# Carregando um dataset
dataset = load_wine()
x = dataset.data
y = dataset.target

# Criando um obj KMeans e fazendo um agrupamento de TODOS os dados com três clusters, como no dataset original
objKMeans = KMeans(n_clusters=3)
objKMeans.fit(x)

# Verificando o WB Index da clusterização. A função criada deve receber x, y e os centroides de cada cluster.
indexWB = WBIndex(x, objKMeans.labels_, objKMeans.cluster_centers_)

# Printando
print("WB Index com 2 clusters:", indexWB)

WB Index com 2 clusters: 27.72261645271297
